In [1]:
%%capture
!pip install pip3-autoremove
!pip-autoremove torch torchvision torchaudio -y
!pip install torch torchvision torchaudio xformers --index-url https://download.pytorch.org/whl/cu121
!pip install unsloth
!pip install --upgrade --no-cache-dir transformers
import os
os.environ["WANDB_DISABLED"] = "true"

In [6]:
from datasets import Dataset

In [2]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.


#Load the model saved after full training
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "minemaster01/qwen2.5-7b-fmd-5-full",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


==((====))==  Unsloth 2024.11.5: Fast Qwen2 patching. Transformers = 4.46.2.
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.5.1+cu121. CUDA = 7.5. CUDA Toolkit = 12.1.
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.28.post3. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/5.55G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/117 [00:00<?, ?B/s]

...

added_tokens.json:   0%|          | 0.00/632 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/616 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/7.03M [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/162M [00:00<?, ?B/s]

Unsloth 2024.11.5 patched 28 layers with 0 QKV layers, 28 O layers and 28 MLP layers.


In [14]:
prompt_template = """
Please label whether the claim is True, False, or Not Enough Information based on claim and its justification. Also provide appropriate explanation as evidence.
### claim: {}
### justification: {}
### output:
"""

In [20]:
import pandas as pd
df=pd.read_json('/content/FINNLP-test.json',lines=True)

In [21]:
import torch
torch._dynamo.config.suppress_errors = True
from tqdm.auto import tqdm

In [29]:
trail_df = df[:1]

In [30]:
for index, row in tqdm(trail_df.iterrows(), total=trail_df.shape[0], desc="Making Predictions"):
  inputs = tokenizer([prompt_template.format(row['claim'],row['justification'])],return_tensors="pt").to("cuda")
  outputs = model.generate(**inputs, max_new_tokens=128, num_return_sequences=1, do_sample=False, eos_token_id=tokenizer.eos_token_id, pad_token_id=tokenizer.pad_token_id,)
  decoded_output = tokenizer.decode(outputs[0], skip_special_tokens=True)
  output_split = decoded_output.split("### output:")
  if len(output_split) > 1:
    response_part = output_split[1].strip()  # Removes leading/trailing whitespace
  else:
    response_part = ""  # Handle cases where "### output:" is not found
  label_split = response_part.split("label:")
  if len(label_split) > 1:
    ans_label = label_split[1].split("\n")[0].strip()
  else:
    ans_label = None
  evidence_split = response_part.split("evidence:")
  if len(evidence_split) > 1:
    ans_expln = evidence_split[1].strip()
  else:
    ans_expln = None
  print(f"Label: {ans_label}")
  print(f"Explanation: {ans_expln}")

Making Predictions:   0%|          | 0/1 [00:00<?, ?it/s]

Label: None
Explanation: None


In [23]:
df['pred'] = None
df['explanation'] = None
for index, row in tqdm(df.iterrows(), total=df.shape[0], desc="Making Predictions"):
  inputs = tokenizer([prompt_template.format(row['claim'],row['justification'])],return_tensors="pt").to("cuda")
  outputs = model.generate(**inputs, max_new_tokens=128, num_return_sequences=1, do_sample=False, eos_token_id=tokenizer.eos_token_id, pad_token_id=tokenizer.pad_token_id,)
  decoded_output = tokenizer.decode(outputs[0], skip_special_tokens=True)
  output_split = decoded_output.split("### output:")
  if len(output_split) > 1:
    response_part = output_split[1].strip()  # Removes leading/trailing whitespace
  else:
    response_part = ""  # Handle cases where "### output:" is not found
  label_split = response_part.split("label:")
  if len(label_split) > 1:
    ans_label = label_split[1].split("\n")[0].strip()
  else:
    ans_label = None
  evidence_split = response_part.split("evidence:")
  if len(evidence_split) > 1:
    ans_expln = evidence_split[1].strip()
  else:
    ans_expln = None
  df.at[index, 'pred'] = ans_label
  df.at[index, 'explanation'] = ans_expln

Making Predictions:   0%|          | 0/1304 [00:00<?, ?it/s]

In [24]:
submission_df = df[['ID', 'pred', 'explanation']].copy()

In [26]:
submission_df

,ID,pred,explanation
0,FMD_test_0,None,None
1,FMD_test_1,None,None
2,FMD_test_2,None,None
3,FMD_test_3,None,None
4,FMD_test_4,None,None
...,...,...,...
1299,FMD_test_1299,None,None
1300,FMD_test_1300,None,None
1301,FMD_test_1301,None,None
1302,FMD_test_1302,None,None


In [25]:
submission_df.to_csv('Qwen-submission-5.csv', index=False)